In [1]:
import pandas as pd
from datetime import date, timedelta

In [2]:
file_path = "https://www.dhhs.vic.gov.au/ncov-covid-cases-by-age-group-csv"
df = pd.read_csv(file_path)

In [3]:
df.tail()

,diagnosis_date,agegroup
649825,2022-02-22,0-9
649826,2022-02-22,50-59
649827,2022-02-22,0-9
649828,2022-02-22,70-79
649829,2022-02-22,10-19_


In [4]:
df_pivot = pd.pivot_table(df, index = 'diagnosis_date', columns = 'agegroup', 
                         values = 'agegroup', aggfunc = len, fill_value = 0)

In [5]:
df_pivot.tail()

agegroup,0-9,10-19_,20-29,30-39,40-49,50-59,60-69,70-79,80-89,90+
diagnosis_date,,,,,,,,,,
2022-02-18,373,395,297,407,372,179,116,72,31,12
2022-02-19,329,300,237,358,291,152,88,55,16,6
2022-02-20,304,349,179,297,251,130,85,41,24,13
2022-02-21,338,375,244,333,295,153,84,56,29,6
2022-02-22,419,434,310,440,370,178,153,73,37,8


In [6]:
# check that the latest date's sum matches the number of cases reported by Vic govt
# the latest row's date is always yesterday

yesterday = date.today() - timedelta(days = 1)

# the dates in the pivot table are strings
# so use isoformat() to convert date into string before looking it up in the table
df_pivot.loc[yesterday.isoformat()].sum()

2422

In [7]:
# take only the 0-9 and 10-19 cases, from 5 Dec onwards
# for each date, store the previous date's cases; this matches what I've done for NSW
# eg. on 8 Dec, the Vic cases are those reported on the 7th

first = date(2021, 12, 5)
today = date.today()

df_0_19 = pd.DataFrame(columns = ['date', 'cases_0_9', 'cases_10_19'])

curr = first
prev = first - timedelta(days = 1)

while (curr < today):
    
        df_0_19 = df_0_19.append({'date': curr.isoformat(), 'cases_0_9': df_pivot.loc[prev.isoformat(), '0-9'], 
                                  'cases_10_19': df_pivot.loc[prev.isoformat(), '10-19_']}, ignore_index = True)
        prev = curr
        curr = curr + timedelta(days = 1)

In [8]:
df_0_19.tail()

,date,cases_0_9,cases_10_19
75,2022-02-18,423,483
76,2022-02-19,373,395
77,2022-02-20,329,300
78,2022-02-21,304,349
79,2022-02-22,338,375


In [9]:
df_0_19.to_csv("../data/cases_0_19.csv")